In [ ]:
!pip install -q torchaudio
!pip install torch
!pip install -q pydub
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0
USE_ONNX = False # change this to True if you want to test onnx model
if USE_ONNX:
    !pip install -q onnxruntime
!pip install -q huggingsound
#@title Install and Import Dependencies

# this assumes that you have a relevant version of PyTorch installed
SAMPLING_RATE = 16000
import torch
torch.set_num_threads(1)
from IPython.display import Audio
from pprint import pprint
# download example
# torch.hub.download_url_to_file('https://models.silero.ai/vad_models/en.wav', 'en_example.wav')
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import re
import json
import torch
import torch.nn as n
import transformers
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from datasets import Dataset
from peft import LoraConfig, PeftConfig
import bitsandbytes as bnb
from trl import SFTTrainer
from tqdm.notebook import tqdm
# from sklearn.metrics import (accuracy_score,
#                              classification_report,
#                              confusion_matrix)
# from sklearn.model_selection import train_test_split
from datasets import load_dataset
import os
os.environ["WANDB_DISABLED"] = "true"


model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True,
                              onnx=USE_ONNX)

(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils

model_name = " qwen2-7b-instruct"

compute_dtype = getattr(torch, "float16")

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=False,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=compute_dtype,
# )

model_txt = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    # quantization_config=bnb_config,
    local_files_only = True
)

model_txt.config.use_cache = False
model_txt.config.pretraining_tp = 1

tokenizer_txt = AutoTokenizer.from_pretrained(model_name)
## using VADIterator class
# ! ffmpeg -i /content/voice_11318.mp4 -acodec pcm_s16le -ar 16000 -y output.wav

from pydub import AudioSegment
import os
from huggingsound import SpeechRecognitionModel
import torch
import transformers
vad_iterator = VADIterator(model)
timestamps = []
wav = read_audio(f'/content/output.wav', sampling_rate=SAMPLING_RATE)

window_size_samples = 1536 # number of samples in a single audio chunk
for i in range(0, len(wav), window_size_samples):
    chunk = wav[i: i+ window_size_samples]
    if len(chunk) < window_size_samples:
      break
    speech_dict = vad_iterator(chunk, return_seconds=True)
    if speech_dict:
        timestamps.append(speech_dict)
        # print(speech_dict, end=' ')
vad_iterator.reset_states() # reset model states after each audio


def cut_wav_chunks(input_file, timestamps):
    # Load the audio file
    audio = AudioSegment.from_wav(input_file)

    chunks = []
    for i in range(0, len(timestamps), 2):
        start = timestamps[i].get('start', 0) * 1000  # Convert to milliseconds
        end = timestamps[i+1].get('end', len(audio)) * 1000  # Convert to milliseconds
        chunk = audio[start:end]
        chunks.append(chunk)

    return chunks
# Path to your input WAV file
input_file = "/content/output.wav"
output_dir = "/content/sound"
os.makedirs(output_dir, exist_ok=True)
# Cut the chunks
chunks = cut_wav_chunks(input_file, timestamps)

# Save the chunks
for i, chunk in enumerate(chunks):

    chunk.export(f"/content/sound/chunk_{i+1}.wav", format="wav")

# Function to extract the numeric value from the filename


audio_path = []
for a in os.listdir('/content/sound'):
  full_path = os.path.join('/content/sound', a)
  if os.path.isfile(full_path):
    audio_path.append(full_path)


def extract_number(file_path):
    filename = file_path.split('/')[-1]
    number = int(filename.split('_')[1].split('.')[0])
    return number

sorted_file_paths = sorted(audio_path, key=extract_number)



device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = SpeechRecognitionModel("airesearch/wav2vec2-large-xlsr-53-th", device = device)

transcriptions = model.transcribe(sorted_file_paths)
chunk_word = []
full_transcript =''
for item in transcriptions:
    chunk_word.append(item['transcription'].replace(" ",""))
    print("//",item['transcription'].replace(" ",""),"//")
    full_transcript += ''.join(item['transcription'].replace(" ",""))


def extract_and_combine(text):
    """
    Extract JSON strings between |<start>| and |<end>| markers and combine them into a single JSON array.

    :param text: The input text containing the JSON strings
    :return: A combined JSON string
    """
    trl_ar = []
    # Regular expression to find JSON strings between |<start>| and |<end>|
    pattern = r'\|<start>\|(.*?)\|<end>\|'
    matches = re.findall(pattern, text)
    [trl_ar.append(matche) for matche in matches]
    # # Parse the matched JSON strings and combine them into a list
    # json_objects = [json.loads(match) for match in matches]
    
    # # Convert the list of JSON objects back to a JSON string
    # combined_json_string = json.dumps(json_objects, ensure_ascii=False)
    
    return trl_ar

def cutting(prompt):
    messagee = prompt.split(" ")
    return messagee


tokenizer.pad_token_id = tokenizer.eos_token_id
pipeline = transformers.pipeline(
    "text-generation", model=model_txt,
    tokenizer=tokenizer,
    model_kwargs={"torch_dtype": torch.float16},
    trust_remote_code = True,
    device_map="auto",
)

if full_transcript:
    prompt = full_transcript
else:
    prompt = """มีการรายงานข่าวเกี่ยวกับเหตุการณ์ที่เจ้าหน้าที่ตำรวจเข้าตรวจสอบอาจารย์หญิงคนหนึ่งที่เป็นผู้นำทางจิตวิญญาณของชาวบ้านในสำนักสุขาวดี จังหวัดอุดรธานี โดยตอนแรกเจ้าหน้าที่ไม่ทราบชื่อจริงของเธอ จนกระทั่งพบว่าเธอเคยเปลี่ยนชื่อมาหลายครั้ง และมีหมายจับเกี่ยวกับการทุจริตและการนำเข้าข้อมูลอันเป็นเท็จสู่ระบบคอมพิวเตอร์เมื่อเจ้าหน้าที่ควบคุมตัวอาจารย์หญิงคนนี้ เธอกล่าวว่าไม่ได้รู้สึกเครียดและบอกว่าร่างของเธอถูกครอบครองโดยวิญญาณอื่น จากการตรวจสอบพบว่าเธอเคยเป็นนักธุรกิจและซีอีโอของบริษัทหนึ่ง ซึ่งหลังจากมีคดีความเกี่ยวกับการเงิน เธอก็หันมาเป็นผู้นำทางจิตวิญญาณในห้องขัง เธอนั่งสมาธิและอ้างว่าได้ช่วยปลดปล่อยดวงวิญญาณนักรบในอดีตหลายหมื่นดวง อย่างไรก็ตาม เจ้าหน้าที่ตำรวจยืนยันว่าเธอมีคดีความที่ยังต้องสอบสวนและมีการควบคุมตัวต่อไป ซึ่งเธอได้รับการประกันตัวในที่สุดเหตุการณ์นี้ทำให้หลายคนสงสัยเกี่ยวกับความเชื่อและพฤติกรรมของเธอ รวมถึงการดำเนินการของเจ้าหน้าที่ในการตรวจสอบคดีความของเธอที่เกี่ยวข้องกับการทุจริตในอดีต"""

messagee = cutting(prompt)
persona_llm = """you are Narin Phon an experrt in correcting sentence
Skills:
Text Correction: Expertise in identifying and correcting grammatical, syntactical, and contextual errors in Thai text.
Data Annotation: Experience in creating and managing annotated datasets for training language models."""
massive = []
for i in messagee:
    txt_messages = [
        {"role": "system", "content": f"""{persona_llm}.
        you must answer like this [
            input =  พระบรมมทาตสวีตั้งอยู่บนถนนเพชรเกษมที่ตั้งท่ามกลางทะเลอันดาหมั่นของจังหวัดสงขลานมีตำนานเกี่ยวกับอารังกาศักดิ์สิทธิิ์ซึ่งรวมอยู่ในภูเหารับล่อใกล้เคียงขับ
            output = |<start>|พระบรมธาตุสวีตั้งอยู่บนถนนเพชรเกษมที่ตั้งท่ามกลางทะเลอันดามันของจังหวัดสงขลา มีตำนานเกี่ยวกับลังกาศักดิ์สิทธิ์ซึ่งร่วมอยู่ในภูเขารับร่อใกล้เคียงครับ|<end>|
            ]
            if output has more than one use "," between them 
        """},
        {"role": "user", "content":f"{i}"}
    ]
    massive.append(txt_messages)

outputs = []
for j in tqdm(range(len(messagee))):
    i = messagee[j]
    txt_messages = massive[j]
    out = pipeline(
        txt_messages,
        max_new_tokens=128,
        temperature=0.01,
        early_stopping=True,
        num_beams=1,top_k=3, 
        eos_token_id=model.config.eos_token_id
    )
    # print(out)
    outputs.append(out)


cleaned = []
for i in outputs:
    output_text = extract_and_combine(i[0]["generated_text"][2]["content"])
    cleaned.append(output_text[0])
# df = to_dfa(cleaned)
# df
correct_txt = " ".join(cleaned)
correct_txt

In [ ]:
#@title Install and Import Dependencies

# this assumes that you have a relevant version of PyTorch installed
SAMPLING_RATE = 16000
import torch
torch.set_num_threads(1)
from IPython.display import Audio
from pprint import pprint
# download example
# torch.hub.download_url_to_file('https://models.silero.ai/vad_models/en.wav', 'en_example.wav')
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

import torch
import torch.nn as nn

import transformers
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from datasets import Dataset
from peft import LoraConfig, PeftConfig
import bitsandbytes as bnb
from trl import SFTTrainer
from tqdm.notebook import tqdm
# from sklearn.metrics import (accuracy_score,
#                              classification_report,
#                              confusion_matrix)
# from sklearn.model_selection import train_test_split
from datasets import load_dataset
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:

model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True,
                              onnx=USE_ONNX)

(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils

In [ ]:
## using VADIterator class
# ! ffmpeg -i /content/voice_11318.mp4 -acodec pcm_s16le -ar 16000 -y output.wav

from pydub import AudioSegment
import os

vad_iterator = VADIterator(model)
timestamps = []
wav = read_audio(f'/content/output.wav', sampling_rate=SAMPLING_RATE)

window_size_samples = 1536 # number of samples in a single audio chunk
for i in range(0, len(wav), window_size_samples):
    chunk = wav[i: i+ window_size_samples]
    if len(chunk) < window_size_samples:
      break
    speech_dict = vad_iterator(chunk, return_seconds=True)
    if speech_dict:
        timestamps.append(speech_dict)
        # print(speech_dict, end=' ')
vad_iterator.reset_states() # reset model states after each audio


def cut_wav_chunks(input_file, timestamps):
    # Load the audio file
    audio = AudioSegment.from_wav(input_file)

    chunks = []
    for i in range(0, len(timestamps), 2):
        start = timestamps[i].get('start', 0) * 1000  # Convert to milliseconds
        end = timestamps[i+1].get('end', len(audio)) * 1000  # Convert to milliseconds
        chunk = audio[start:end]
        chunks.append(chunk)

    return chunks
# Path to your input WAV file
input_file = "/content/output.wav"
output_dir = "/content/sound"
os.makedirs(output_dir, exist_ok=True)
# Cut the chunks
chunks = cut_wav_chunks(input_file, timestamps)

# Save the chunks
for i, chunk in enumerate(chunks):

    chunk.export(f"/content/sound/chunk_{i+1}.wav", format="wav")

In [ ]:
# Function to extract the numeric value from the filename


audio_path = []
for a in os.listdir('/content/sound'):
  full_path = os.path.join('/content/sound', a)
  if os.path.isfile(full_path):
    audio_path.append(full_path)


def extract_number(file_path):
    filename = file_path.split('/')[-1]
    number = int(filename.split('_')[1].split('.')[0])
    return number

sorted_file_paths = sorted(audio_path, key=extract_number)

In [ ]:
from huggingsound import SpeechRecognitionModel
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = SpeechRecognitionModel("airesearch/wav2vec2-large-xlsr-53-th", device = device)

transcriptions = model.transcribe(sorted_file_paths)

In [ ]:


chunk_word = []
full_transcript =''
for item in transcriptions:
    chunk_word.append(item['transcription'].replace(" ",""))
    print("//",item['transcription'].replace(" ",""),"//")
    full_transcript += ''.join(item['transcription'].replace(" ",""))

In [ ]:
full_transcript

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.4.1 requires cubinlinker, which is not installed.
cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.4.1 requires ptxcompiler, which is not installed.
cuml 24.4.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.4.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.4.1 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.5.0 which is incompatible.
distributed 2024.1.1 requires dask==2024.1.1, but you have dask 2024.5.2 which is incompatible.
gcsfs 2024.3.1 requires fsspec==2024.3.1, but you have fsspec 2023.10.0 which is incompatible.
rapids-dask-dependency 24.4.1a0 requires dask==2024.1.1, but you have dask 2024.5.2 which is incompatible.
rapids-dask-dependency 24.4.1a0 requires dask-expr==0.4.0, but you have dask-expr 1.1.2 which is inc

In [2]:
model_name = "model/qwen2-72b"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    local_files_only = True
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/37 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# train = pd.read_csv("text2cypher_gpt4o.csv")
# train1 = train[["question","cypher"]][:2]
# train1
# .to_csv("train2.csv",index = False)


SyntaxError: invalid syntax (712746375.py, line 4)

In [8]:
# ls = []
# for i in tqdm(range(len(train))):
#     row = train.iloc[i]
#     prompt = {
#         "messages" :[
#             {"role": "system", "content": "You are an expert knowledge graph annotator and you respond in JSON. Here's the json schema you must adhere to where each element is a new triple if needed"},
#             {"role": "user", "content": f'transfrom this to cypher {row["question"]}'},
#             {'role': 'assistant', 'content': f'{row["cypher"]}'}
#         ]
#     }
#     ls.append(prompt)

  0%|          | 0/9468 [00:00<?, ?it/s]

In [9]:
# import json
# with open("train_full.json", "w") as json_file:
#     json.dump(ls, json_file)

In [5]:
# train_dataset = load_dataset("json",data_files = "train_full.json",split="train").train_test_split(test_size=0.2)


In [ ]:
# train_dataset["train"][:100]

In [42]:
# peft_config = LoraConfig(
#     lora_alpha=16,
#     r=64,
#     bias="none",
#     task_type="CAUSAL_LM",
#     target_modules=["q_proj",
#             "k_proj",
#             "v_proj",
#             "o_proj",
#             "up_proj",
#             "gate_proj",
#             "down_proj",]
# )

# training_arguments = TrainingArguments(
#     output_dir="logs",
#     num_train_epochs=2,
#     per_device_train_batch_size=1,
#     gradient_accumulation_steps=8,
#     gradient_checkpointing=False,
#     optim="paged_adamw_32bit",
#     save_steps=0,
#     logging_steps=25,
#     learning_rate=2e-1,
#     weight_decay=0.001,
#     fp16=True,
#     bf16=False,
#     max_grad_norm=0.3,
#     max_steps=-1,
#     warmup_ratio=0.03,
#     group_by_length=True,
#     lr_scheduler_type="cosine",
#     # report_to="tensorboard",
#     do_eval=False,
#     evaluation_strategy="no",
# )

# trainer = SFTTrainer(
#     model=model,
#     train_dataset=train_dataset["train"],
#     peft_config=peft_config,
#     tokenizer=tokenizer,
#     args=training_arguments,
#     packing=False,
#     max_seq_length=512,

# )

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Map:   0%|          | 0/7574 [00:00<?, ? examples/s]

In [ ]:
# trainer.train()

In [25]:
# train_dataset["train"][0]

{'messages': [{'content': "You are an expert knowledge graph annotator and you respond in JSON. Here's the json schema you must adhere to where each element is a new triple if needed",
   'role': 'system'},
  {'content': 'transfrom this to cypher Which organizations in New York City have the most employees, list the top 3.',
   'role': 'user'},
  {'content': 'MATCH (o:Organization)-[:IN_CITY]->(c:City {name: "New York City"})\nRETURN o.name, o.nbrEmployees\nORDER BY o.nbrEmployees DESC\nLIMIT 3',
   'role': 'assistant'}]}

In [3]:
import transformers
tokenizer.pad_token_id = tokenizer.eos_token_id
pipeline = transformers.pipeline(
    "text-generation", model=model,
    tokenizer=tokenizer,
    model_kwargs={"torch_dtype": torch.float16},
    trust_remote_code = True,
    device_map="auto",
)

if full_transcript:
    prompt = full_transcript
else:
    prompt = """มีการรายงานข่าวเกี่ยวกับเหตุการณ์ที่เจ้าหน้าที่ตำรวจเข้าตรวจสอบอาจารย์หญิงคนหนึ่งที่เป็นผู้นำทางจิตวิญญาณของชาวบ้านในสำนักสุขาวดี จังหวัดอุดรธานี โดยตอนแรกเจ้าหน้าที่ไม่ทราบชื่อจริงของเธอ จนกระทั่งพบว่าเธอเคยเปลี่ยนชื่อมาหลายครั้ง และมีหมายจับเกี่ยวกับการทุจริตและการนำเข้าข้อมูลอันเป็นเท็จสู่ระบบคอมพิวเตอร์เมื่อเจ้าหน้าที่ควบคุมตัวอาจารย์หญิงคนนี้ เธอกล่าวว่าไม่ได้รู้สึกเครียดและบอกว่าร่างของเธอถูกครอบครองโดยวิญญาณอื่น จากการตรวจสอบพบว่าเธอเคยเป็นนักธุรกิจและซีอีโอของบริษัทหนึ่ง ซึ่งหลังจากมีคดีความเกี่ยวกับการเงิน เธอก็หันมาเป็นผู้นำทางจิตวิญญาณในห้องขัง เธอนั่งสมาธิและอ้างว่าได้ช่วยปลดปล่อยดวงวิญญาณนักรบในอดีตหลายหมื่นดวง อย่างไรก็ตาม เจ้าหน้าที่ตำรวจยืนยันว่าเธอมีคดีความที่ยังต้องสอบสวนและมีการควบคุมตัวต่อไป ซึ่งเธอได้รับการประกันตัวในที่สุดเหตุการณ์นี้ทำให้หลายคนสงสัยเกี่ยวกับความเชื่อและพฤติกรรมของเธอ รวมถึงการดำเนินการของเจ้าหน้าที่ในการตรวจสอบคดีความของเธอที่เกี่ยวข้องกับการทุจริตในอดีต"""

In [50]:
full_transcript = ""

In [51]:
bool(full_transcript)

False

In [73]:
if full_transcript:
    prompt = full_transcript
else:
    prompt = """มีการรายงานข่าวเกี่ยวกับเหตุการณ์ที่เจ้าหน้าที่ตำรวจเข้าตรวจสอบอาจารย์หญิงคนหนึ่งที่เป็นผู้นำทางจิตวิญญาณของชาวบ้านในสำนักสุขาวดี จังหวัดอุดรธานี โดยตอนแรกเจ้าหน้าที่ไม่ทราบชื่อจริงของเธอ จนกระทั่งพบว่าเธอเคยเปลี่ยนชื่อมาหลายครั้ง และมีหมายจับเกี่ยวกับการทุจริตและการนำเข้าข้อมูลอันเป็นเท็จสู่ระบบคอมพิวเตอร์เมื่อเจ้าหน้าที่ควบคุมตัวอาจารย์หญิงคนนี้ เธอกล่าวว่าไม่ได้รู้สึกเครียดและบอกว่าร่างของเธอถูกครอบครองโดยวิญญาณอื่น จากการตรวจสอบพบว่าเธอเคยเป็นนักธุรกิจและซีอีโอของบริษัทหนึ่ง ซึ่งหลังจากมีคดีความเกี่ยวกับการเงิน เธอก็หันมาเป็นผู้นำทางจิตวิญญาณในห้องขัง เธอนั่งสมาธิและอ้างว่าได้ช่วยปลดปล่อยดวงวิญญาณนักรบในอดีตหลายหมื่นดวง อย่างไรก็ตาม เจ้าหน้าที่ตำรวจยืนยันว่าเธอมีคดีความที่ยังต้องสอบสวนและมีการควบคุมตัวต่อไป ซึ่งเธอได้รับการประกันตัวในที่สุดเหตุการณ์นี้ทำให้หลายคนสงสัยเกี่ยวกับความเชื่อและพฤติกรรมของเธอ รวมถึงการดำเนินการของเจ้าหน้าที่ในการตรวจสอบคดีความของเธอที่เกี่ยวข้องกับการทุจริตในอดีต"""

In [74]:
def cutting(prompt):
    messagee = prompt.split(" ")
    return messagee
messagee = cutting(prompt)
messagee

['มีการรายงานข่าวเกี่ยวกับเหตุการณ์ที่เจ้าหน้าที่ตำรวจเข้าตรวจสอบอาจารย์หญิงคนหนึ่งที่เป็นผู้นำทางจิตวิญญาณของชาวบ้านในสำนักสุขาวดี',
 'จังหวัดอุดรธานี',
 'โดยตอนแรกเจ้าหน้าที่ไม่ทราบชื่อจริงของเธอ',
 'จนกระทั่งพบว่าเธอเคยเปลี่ยนชื่อมาหลายครั้ง',
 'และมีหมายจับเกี่ยวกับการทุจริตและการนำเข้าข้อมูลอันเป็นเท็จสู่ระบบคอมพิวเตอร์เมื่อเจ้าหน้าที่ควบคุมตัวอาจารย์หญิงคนนี้',
 'เธอกล่าวว่าไม่ได้รู้สึกเครียดและบอกว่าร่างของเธอถูกครอบครองโดยวิญญาณอื่น',
 'จากการตรวจสอบพบว่าเธอเคยเป็นนักธุรกิจและซีอีโอของบริษัทหนึ่ง',
 'ซึ่งหลังจากมีคดีความเกี่ยวกับการเงิน',
 'เธอก็หันมาเป็นผู้นำทางจิตวิญญาณในห้องขัง',
 'เธอนั่งสมาธิและอ้างว่าได้ช่วยปลดปล่อยดวงวิญญาณนักรบในอดีตหลายหมื่นดวง',
 'อย่างไรก็ตาม',
 'เจ้าหน้าที่ตำรวจยืนยันว่าเธอมีคดีความที่ยังต้องสอบสวนและมีการควบคุมตัวต่อไป',
 'ซึ่งเธอได้รับการประกันตัวในที่สุดเหตุการณ์นี้ทำให้หลายคนสงสัยเกี่ยวกับความเชื่อและพฤติกรรมของเธอ',
 'รวมถึงการดำเนินการของเจ้าหน้าที่ในการตรวจสอบคดีความของเธอที่เกี่ยวข้องกับการทุจริตในอดีต']

In [ ]:
# persona_cypher = """Name: Alex Cipher

# Title: Data Scientist and Cypher Query Specialist

# Skills:

# Natural Language Processing (NLP): Proficient in transforming natural language text into structured data.
# Graph Databases: Expert in Neo4j and other graph database technologies.
# Cypher Query Language: Advanced knowledge and extensive experience in writing and optimizing Cypher queries.
# Machine Learning: Familiar with machine learning techniques for text analysis and conversion.
# Programming Languages: Skilled in Python, with specific libraries such as py2neo and networkx for interacting with graph databases.
# Data Parsing and Transformation: Expertise in parsing text data and transforming it into query-friendly formats."""

In [8]:
# mas = []
# for i in messagee:
#     txt_messages = [
#         {"role": "system", "content": """You are an expert knowledge graph annotator and you respond in JSON and respond only the querry.
#         Here's the json schema you must adhere to where each element is a new triple if needed.
#         you must answer like this [
#             input =  วัดเทพเจริญ เขารับร่อ
#             output = |<start>|{"subject": "วัดเทพเจริญ", "predicate": "LOCATION", "object": "เขารับร่อ"}|<end>|
#             input =  วัดเทพเจริญ คลองรับร่อ
#             output = |<start>|{"subject": "วัดเทพเจริญ", "predicate": "LOCATION", "object": "คลองรับร่อ"}|<end>|
#             input =  วัดเทพเจริญ เมืองนคร
#             output = |<start>|{"subject": "วัดเทพเจริญ", "predicate": "LOCATION", "object": "เมืองนคร"}}|<end>|
#             input =  วัดเทพเจริญ สมัยอยุธยา
#             output = |<start>|{"subject": "วัดเทพเจริญ", "predicate": "DATE", "object": "สมัยอยุธยา"}|<end>|
#             input =  วัดเทพเจริญ ถ้ำรับร่อ
#             output = |<start>|{"subject": "วัดเทพเจริญ", "predicate": "PLACE", "object": "ถ้ำรับร่อ"|<end>|
#             input = นายจันทร์เรียนอยู่ที่วัดเทพเจริญ ณ เวลา 15.30 น.
#             output = |<start>|{"subject": "นายจันทร์", "predicate": "STUDY", "object": "วัดเทพเจริญ"}|<end>|,|<start>|{"subject": "นายจันทร์", "predicate": "TIME", "object": "15.30 น."}|<end>|
    
#             ]
#             if output has more than one use "," between them 
#         """},
#         {"role": "user", "content":f"{i}"}
#     ]
#     mas.append(txt_messages)



In [16]:
persona_llm = """you are Narin Phon an experrt in correcting sentence
Skills:
Text Correction: Expertise in identifying and correcting grammatical, syntactical, and contextual errors in Thai text.
Data Annotation: Experience in creating and managing annotated datasets for training language models."""
massive = []
for i in messagee:
    txt_messages = [
        {"role": "system", "content": f"""{persona_llm}.
        you must answer like this [
            input =  พระบรมมทาตสวีตั้งอยู่บนถนนเพชรเกษมที่ตั้งท่ามกลางทะเลอันดาหมั่นของจังหวัดสงขลานมีตำนานเกี่ยวกับอารังกาศักดิ์สิทธิิ์ซึ่งรวมอยู่ในภูเหารับล่อใกล้เคียงขับ
            output = |<start>|พระบรมธาตุสวีตั้งอยู่บนถนนเพชรเกษมที่ตั้งท่ามกลางทะเลอันดามันของจังหวัดสงขลา มีตำนานเกี่ยวกับลังกาศักดิ์สิทธิ์ซึ่งร่วมอยู่ในภูเขารับร่อใกล้เคียงครับ|<end>|
            ]
            if output has more than one use "," between them 
        """},
        {"role": "user", "content":f"{i}"}
    ]
    massive.append(txt_messages)



In [18]:
def get_token(word):
    """
    Get the token of a word using a specified tokenizer model from Hugging Face.

    :param word: The word to be tokenized
    :param model_name: The name of the pre-trained model to use for tokenization
    :return: List of tokens
    """
    # Initialize the tokenizer
    
    # Tokenize the word
    tokens = tokenizer.tokenize(word)
    
    return len(tokens)

In [25]:
outputs = []
for j in tqdm(range(len(messagee))):
    i = messagee[j]
    txt_messages = massive[j]
    out = pipeline(
        txt_messages,
        max_new_tokens=128,
        temperature=0.01,
        early_stopping=True,
        num_beams=1,top_k=3, 
        eos_token_id=model.config.eos_token_id
    )
    # print(out)
    outputs.append(out)

  0%|          | 0/14 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
/lustrefs/disk/project/ck900819-ckos19/loader/ida-env/lib/python3.12/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pa

In [26]:
out

[{'generated_text': [{'role': 'system',
    'content': 'you are Narin Phon an experrt in correcting sentence\nSkills:\nText Correction: Expertise in identifying and correcting grammatical, syntactical, and contextual errors in Thai text.\nData Annotation: Experience in creating and managing annotated datasets for training language models..\n        you must answer like this [\n            input =  พระบรมมทาตสวีตั้งอยู่บนถนนเพชรเกษมที่ตั้งท่ามกลางทะเลอันดาหมั่นของจังหวัดสงขลานมีตำนานเกี่ยวกับอารังกาศักดิ์สิทธิิ์ซึ่งรวมอยู่ในภูเหารับล่อใกล้เคียงขับ\n            output = |<start>|พระบรมธาตุสวีตั้งอยู่บนถนนเพชรเกษมที่ตั้งท่ามกลางทะเลอันดามันของจังหวัดสงขลา มีตำนานเกี่ยวกับลังกาศักดิ์สิทธิ์ซึ่งร่วมอยู่ในภูเขารับร่อใกล้เคียงครับ|<end>|\n            ]\n            if output has more than one use "," between them \n        '},
   {'role': 'user',
    'content': 'รวมถึงการดำเนินการของเจ้าหน้าที่ในการตรวจสอบคดีความของเธอที่เกี่ยวข้องกับการทุจริตในอดีต'},
   {'role': 'assistant',
    'content': '

In [76]:
import re
import json

def extract_and_combine(text):
    """
    Extract JSON strings between |<start>| and |<end>| markers and combine them into a single JSON array.

    :param text: The input text containing the JSON strings
    :return: A combined JSON string
    """
    trl_ar = []
    # Regular expression to find JSON strings between |<start>| and |<end>|
    pattern = r'\|<start>\|(.*?)\|<end>\|'
    matches = re.findall(pattern, text)
    [trl_ar.append(matche) for matche in matches]
    # # Parse the matched JSON strings and combine them into a list
    # json_objects = [json.loads(match) for match in matches]
    
    # # Convert the list of JSON objects back to a JSON string
    # combined_json_string = json.dumps(json_objects, ensure_ascii=False)
    
    return trl_ar

# Example usage

# def to_dfa(cleaned):
#     sj = []
#     pt = []
#     oj = []
#     for st in cleaned:
#         for sub_st in st:
#             pattern = r'"subject": "(.*?)"'
#             subject = re.findall(pattern, sub_st)
#             pattern = r'"predicate": "(.*?)"'
#             predicate = re.findall(pattern, sub_st)
#             pattern = r'"object": "(.*?)"'
#             objecta = re.findall(pattern, sub_st)
#             sj.append(subject[0])
#             pt.append(predicate[0])
#             oj.append(objecta[0])

#     dicta = {"subject":sj,
#         "predicate":pt,
#         "object":oj}
    # return pd.DataFrame(dicta)


cleaned = []
for i in outputs:
    output_text = extract_and_combine(i[0]["generated_text"][2]["content"])
    cleaned.append(output_text[0])
# df = to_dfa(cleaned)
# df
correct_txt = " ".join(cleaned)
correct_txt

'มีการรายงานข่าวเกี่ยวกับเหตุการณ์ที่เจ้าหน้าที่ตำรวจเข้าตรวจสอบอาจารย์หญิงคนหนึ่งที่เป็นผู้นำทางจิตวิญญาณของชาวบ้านในสำนักสุขาวดีครับ จังหวัดอุดรธานี โดยตอนแรกเจ้าหน้าที่ไม่ทราบชื่อจริงของเธอ จนกระทั่งพบว่าเธอเคยเปลี่ยนชื่อมาหลายครั้งครับ และมีหมายจับเกี่ยวกับการทุจริตและการนำเข้าข้อมูลอันเป็นเท็จสู่ระบบคอมพิวเตอร์ เมื่อเจ้าหน้าที่ควบคุมตัวอาจารย์หญิงคนนี้ เธอกล่าวว่าไม่ได้รู้สึกเครียดและบอกว่าร่างของเธอถูกครอบครองโดยวิญญาณอื่น จากการตรวจสอบพบว่าเธอเคยเป็นนักธุรกิจและซีอีโอของบริษัทหนึ่ง ซึ่งหลังจากมีคดีความเกี่ยวกับการเงิน เธอก็หันมาเป็นผู้นำทางจิตวิญญาณ ในห้องขัง เธอนั่งสมาธิและอ้างว่าได้ช่วยปลดปล่อยดวงวิญญาณนักรบในอดีตหลายหมื่นดวง อย่างไรก็ตาม เจ้าหน้าที่ตำรวจยืนยันว่าเธอมีคดีความที่ยังต้องสอบสวนและมีการควบคุมตัวต่อไป ซึ่งเธอได้รับการประกันตัวในที่สุด เหตุการณ์นี้ทำให้หลายคนสงสัยเกี่ยวกับความเชื่อและพฤติกรรมของเธอ รวมถึงการดำเนินการของเจ้าหน้าที่ในการตรวจสอบคดีความของเธอที่เกี่ยวข้องกับการทุจริตในอดีต'

'มีการรายงานข่าวเกี่ยวกับเหตุการณ์ที่เจ้าหน้าที่ตำรวจเข้าตรวจสอบอาจารย์หญิงคนหนึ่งที่เป็นผู้นำทางจิตวิญญาณของชาวบ้านในสำนักสุขาวดีครับ จังหวัดอุดรธานี โดยตอนแรกเจ้าหน้าที่ไม่ทราบชื่อจริงของเธอ จนกระทั่งพบว่าเธอเคยเปลี่ยนชื่อมาหลายครั้งครับ และมีหมายจับเกี่ยวกับการทุจริตและการนำเข้าข้อมูลอันเป็นเท็จสู่ระบบคอมพิวเตอร์ เมื่อเจ้าหน้าที่ควบคุมตัวอาจารย์หญิงคนนี้ เธอกล่าวว่าไม่ได้รู้สึกเครียดและบอกว่าร่างของเธอถูกครอบครองโดยวิญญาณอื่น จากการตรวจสอบพบว่าเธอเคยเป็นนักธุรกิจและซีอีโอของบริษัทหนึ่ง ซึ่งหลังจากมีคดีความเกี่ยวกับการเงิน เธอก็หันมาเป็นผู้นำทางจิตวิญญาณ ในห้องขัง เธอนั่งสมาธิและอ้างว่าได้ช่วยปลดปล่อยดวงวิญญาณนักรบในอดีตหลายหมื่นดวง อย่างไรก็ตาม เจ้าหน้าที่ตำรวจยืนยันว่าเธอมีคดีความที่ยังต้องสอบสวนและมีการควบคุมตัวต่อไป ซึ่งเธอได้รับการประกันตัวในที่สุด เหตุการณ์นี้ทำให้หลายคนสงสัยเกี่ยวกับความเชื่อและพฤติกรรมของเธอ รวมถึงการดำเนินการของเจ้าหน้าที่ในการตรวจสอบคดีความของเธอที่เกี่ยวข้องกับการทุจริตในอดีต'